In [ ]:
!pip install tensorflow==2.10.0 #tensorflow 2.10.0 <-- 2.12.0 (버전 다운그레이드 필요)

In [1]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

from timeit import default_timer
from transformers import TFGPT2LMHeadModel, AutoTokenizer

print("[모델 로딩 시간]")
start = default_timer()

'''
# We assume the model and the tokenizer has been saved by the below code.
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
'''
loaded_model = TFGPT2LMHeadModel.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")
loaded_tokenizer = AutoTokenizer.from_pretrained("./output/kogpt2-마침표있음-finetuned-epoch-66")

end = default_timer()
print("Time duration(in seconds):", end - start)

def ouputGPT(prompt=""):
    print("[문장 생성 시간]")
    start = default_timer()

    '''
    Temperature: 답변의 창의성과 무작위성을 조정하는 값입니다. 낮을 수록 (예: 0.05) 보다 사실에 근거한 정확한 답변을 제공하고, 높을 수록 (예: 1) 보다 창의적인 결과물을 생성해줍니다. 
    Top_p: 답변의 무작위성을 제어하는 조정 값입니다. Temperature와 마찬가지로 값이 낮을 수록 답변이 보다 정확하고 높아질 수록 창의적이고 광범위해집니다. do_sample=True를 통해 이 중 랜덤한 샘플값을 선택합니다.
    frequency_penalty: 값이 높을 수록 AI가 흔하지 않은 단어를 답변에 포함할 가능성을 제어합니다. 반대로 값이 0이라면 페널티가 전혀 주어지지 않아서 AI가 흔히 사용되지 않는 단어를 답변에 포함할 가능성이 높아지게 됩니다. 
    presence_penalty: 값이 높을 수록 AI가 유사하거나 동일한 단어 및 문구를 답변 시 반복할 가능성을 제어합니다. 이 또한 반대로 값이 0이라면 페널티가 전혀 주어지지 않아서 AI가 단어나 문구를 반복할 가능성이 높아지게 됩니다. 
    '''
    #prompt = "예수님"
    input_ids = loaded_tokenizer.encode(prompt, return_tensors="tf")
    output_ids = loaded_model.generate(input_ids=input_ids,
                                       max_length=30+input_ids.shape[1],
                                       early_stopping=True)
    generated_text = loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    split_generated_text = generated_text.split(prompt)
    if len(split_generated_text) > 1:
        generated_text = split_generated_text[1]
    # Trim the sentences after the last period(.)
    text_to_remove = generated_text.split('.')[-1]
    generated_text = generated_text.replace(text_to_remove,'')

    end = default_timer()
    print("Time duration(in seconds):", end - start)
    return generated_text

# Let's chat for 10 lines
for step in range(10):
    prompt = input(">> User:")
    if prompt.lower() == "bye": break
    print(">> GPT: {}".format( ouputGPT(prompt) ))
    print_current_datetime()

2023-03-31 07:11:50.951797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-31 07:11:50.951827: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gbike/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[모델 로딩 시간]


2023-03-31 07:11:52.989707: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-31 07:11:52.989733: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-31 07:11:52.989753: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-255-120-161): /proc/driver/nvidia/version does not exist
2023-03-31 07:11:52.989955: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 07:11:53.002289: W tensorflow/python/util/util.cc:368] Sets are not currently consider

Time duration(in seconds): 1.968888933013659
>> User:Jesus
[문장 생성 시간]
Time duration(in seconds): 7.126206719025504
>> GPT: . All Rights Reserved.
 @ CDT(2023-03-31T07:12:04.904661)
>> User:BYE


In [1]:
# Microsoft/DialoGPT-large(file size = 3GB) supports multiple languages

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

print_current_datetime("Saving microsoft/DialoGPT-large")

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

tokenizer.save_pretrained("DialoGPT-large-by-Microsoft-Pytorch")
model.save_pretrained("DialoGPT-large-by-Microsoft-Pytorch")

print_current_datetime("Loading DialoGPT-large-by-Microsoft-Pytorch")

tokenizer_loaded = tokenizer.from_pretrained("DialoGPT-large-by-Microsoft-Pytorch")
model_loaded = model.from_pretrained("DialoGPT-large-by-Microsoft-Pytorch")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer_loaded.encode(input(">> User:") + tokenizer_loaded.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model_loaded.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer_loaded.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer_loaded.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

print_current_datetime()

Saving microsoft/DialoGPT-large @ CDT(2023-04-03T09:54:32.972222)
Loading DialoGPT-large-by-Microsoft-Pytorch @ CDT(2023-04-03T09:55:07.304095)
>> User:Hi


2023-04-03 09:55:23.551781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-03 09:55:23.551810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gbike/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


DialoGPT: Hiya
>> User:Do you speak Korean?
DialoGPT: I do
>> User:굉장해요.
DialoGPT: I'm not sure if you're joking or not.
>> User:I see.
DialoGPT: I see what you did there.
>> User:What do you mean?
DialoGPT: I see what you did there.
 @ CDT(2023-04-03T09:56:08.230393)


In [2]:
# Microsoft/DialoGPT-large(file size = 3GB) supports multiple languages
from transformers import AutoTokenizer, TFAutoModelForCausalLM
import tensorflow as tf

from timeit import default_timer

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))
'''
print_current_datetime("Loading pre-trained microsoft/DialoGPT-large")

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

tokenizer.save_pretrained("DialoGPT-large-by-Microsoft-Keras")
model.save_pretrained("DialoGPT-large-by-Microsoft-Keras")
'''
print_current_datetime("Loading saved DialoGPT-large-by-Microsoft-Keras")

loaded_tokenizer = AutoTokenizer.from_pretrained("DialoGPT-large-by-Microsoft-Keras")
loaded_model = TFAutoModelForCausalLM.from_pretrained("DialoGPT-large-by-Microsoft-Keras")

prompt = "Hi"

input_ids = loaded_tokenizer.encode(prompt, return_tensors='tf')
output_ids = loaded_model.generate(
    input_ids,
    max_length=1024,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    num_return_sequences=5, # The model will generate five different responses to the prompt.
    pad_token_id=loaded_tokenizer.eos_token_id
)
print(loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True))

# num_return_sequences=5, which means the model will generate 5 different responses to the prompt.
# The below code loops through the generated responses and print them out with a response number.
for i, chat_history in enumerate(output_ids):
    print(f'Response {i+1}: {loaded_tokenizer.decode(chat_history, skip_special_tokens=True)}')

Loading saved DialoGPT-large-by-Microsoft-Keras @ CDT(2023-04-03T12:08:09.637445)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at DialoGPT-large-by-Microsoft-Keras.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Hi and thank you for the help!
Response 1: Hi and thank you for the help!
Response 2: Hi and welcome to the world of real life.
Response 3: Hi
Response 4: Hi
Response 5: Hi


In [20]:
# Microsoft/DialoGPT-large(file size = 3GB) supports multiple languages
from transformers import AutoTokenizer, TFAutoModelForCausalLM
import tensorflow as tf

from timeit import default_timer

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

loaded_tokenizer = AutoTokenizer.from_pretrained("DialoGPT-large-by-Microsoft-Keras")
loaded_model = TFAutoModelForCausalLM.from_pretrained("DialoGPT-large-by-Microsoft-Keras")

def ouputGPT(prompt=""):
    print("[문장 생성 시간]")
    start = default_timer()

    input_ids = loaded_tokenizer.encode(prompt, return_tensors="tf")
    output_ids = loaded_model.generate(input_ids=input_ids,
                                       max_length=1024,
                                       temperature=0.9,
                                       top_p=0.9,
                                       do_sample=True,
                                       num_return_sequences=1,
                                       pad_token_id=loaded_tokenizer.eos_token_id
                                      )
    generated_text = loaded_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    end = default_timer()
    print("Time duration(in seconds):", end - start)
    return generated_text

# Let's chat for 10 lines
for step in range(10):
    prompt = input(">> User:")
    if prompt.lower() == "bye": break
    print(">> GPT: {}".format( ouputGPT(prompt) ))
    print_current_datetime()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at DialoGPT-large-by-Microsoft-Keras.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


>> User:Hi
[문장 생성 시간]
Time duration(in seconds): 3.3551467680372298
>> GPT: Hi this me :D
 @ CDT(2023-04-03T12:25:59.818459)
>> User:Do you speak Korean?
[문장 생성 시간]
Time duration(in seconds): 0.7616070339572616
>> GPT: Do you speak Korean?
 @ CDT(2023-04-03T12:26:12.380774)
>> User:굉장해요.
[문장 생성 시간]
Time duration(in seconds): 5.190187129017431
>> GPT: 굉장해요.com.in.nl!!
 @ CDT(2023-04-03T12:26:22.268054)
>> User:I see.
[문장 생성 시간]
Time duration(in seconds): 4.091038952989038
>> GPT: I see.You're a real boy
 @ CDT(2023-04-03T12:26:40.649518)
>> User:bye
